In [1]:
import numpy as np
import os
import pandas

In [2]:
filename = os.path.realpath(os.path.join(os.getcwd(), os.path.dirname('treinamento.txt'))) + '/treinamento-1.txt'
series = pandas.read_csv(filename,  header=None)

In [3]:
def series_to_supervised(df, n_lags, n_out=1, dropnan=True):
    """
    Frame a time series as a supervised learning dataset.
    Arguments:
        data: Sequence of observations as a pandas dataframe.
        n_lags: Number of lag observations as input (X).
        n_out: Number of observations as output (y).
        dropnan: Boolean whether or not to drop rows with NaN values.
    Returns:
        Pandas DataFrame of series framed for supervised learning.
    """
    n_vars = df.shape[1]
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_lags, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
    # put it all together
    agg = pandas.concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [4]:
D = series_to_supervised(series, 3).values

X = D[:,0:-1]
Y = D[:,-1].reshape(X.shape[0],1)

train_size = round(X.shape[0] * 0.7)
test_size = X.shape[0] - train_size
Xtr = X[0:train_size,:]
Xv = X[train_size:train_size+test_size,:]
Ytr = Y[0:train_size,:]
Yv = Y[train_size:train_size+test_size,:]

In [5]:
# Xtr = np.array([[0.5348242,0.4279261,0.6337585],
#                  [0.4279261,0.6337585,0.1967004],
#                  [0.6337585,0.1967004,0.9226179],
#                  [0.1967004,0.9226179,-0.7024477],
#                  [0.9226179,-0.7024477,0.0131345],
#                  [-0.7024477,0.0131345,0.999655],
#                  [0.0131345,0.999655,-0.9986200999999999],
#                  [0.999655,-0.9986200999999999,-0.9944841999999998],
#                  [-0.9986200999999999,-0.9944841999999998,-0.9779978000000001],
#                  [-0.9944841999999998,-0.9779978000000001,-0.9129594000000001],
#                  [-0.9779978000000001,-0.9129594000000001,-0.6669898],
#                  [-0.9129594000000001,-0.6669898,0.1102493],
#                  [-0.6669898,0.1102493,0.9756902],
#                  [0.1102493,0.9756902,-0.7811937],
#                  [0.9756902,-0.7811937,0.7368374],
#                  [-0.7811937,0.7368374,-0.7920459],
#                  [0.7368374,-0.7920459,0.6854684],
#                  [-0.7920459,0.6854684,-0.6613866],
#                  [0.6854684,-0.6613866,0.7413643],
#                  [-0.6613866,0.7413643,-0.704547]])

# Ytr = np.array([[0.1967004],
#                 [0.9226179],
#                 [-0.7024477],
#                 [0.0131345],
#                 [0.999655],
#                 [-0.9986200999999999],
#                 [-0.9944841999999998],
#                 [-0.9779978000000001],
#                 [-0.9129594000000001],
#                 [-0.6669898],
#                 [0.1102493],
#                 [0.9756902],
#                 [0.7811937],
#                 [0.7368374],
#                 [-0.7920459],
#                 [0.6854684],
#                 [-0.6613866],
#                 [0.7413643],
#                 [-0.704547],
#                 [0.3774754]])

m = 3

In [6]:
##Pesos para comparacao com Matlab
# W = {}
# W[0] = np.array([[0.93116, 0.72449, 0.59282, 0.70201]])
# W[1] = np.array([[0.27985 , 0.69036, 0.40103, 0.94530]])

# Wg = np.array([[0.34744, 0.45523, 0.99094 , 0.89728],
#        [0.86866, 0.95300, 0.41893, 0.41763 ]])

In [7]:
def softmax(Z):
    Z_exp = np.exp(Z)
    Z_sum = np.sum(Z_exp, axis = 1, keepdims = True)
    return Z_exp/Z_sum   

In [8]:
def maximiza_gating(Wg,Xtr,m,h):
    N = Xtr.shape[0]
    ne = Xtr.shape[1]
    Yg = softmax(np.dot(Xtr,Wg.T))
    
    grad = np.dot((h-Yg).T , (Xtr/N))
    dir = grad
    nit = 0
    nitmax = 50000
    alfa = 0.15
    
    while np.linalg.norm(grad) > 1e-5 and nit < nitmax:
        nit = nit + 1
        Wg = Wg + (alfa * dir) 
        Yg = softmax(np.dot(Xtr,Wg.T))
        grad = np.dot((h-Yg).T , (Xtr/N))
        dir = grad   
        
    return Wg

In [9]:
def maximiza_expert(W,var,Xtr,Ytr,h):
    Ye = np.dot(Xtr,W.T)
    N = Ye.shape[0]
    ns = Ye.shape[1]
    
    grad = np.dot(((h/var) * (Ytr-Ye)).T, Xtr/N)
    
    dir = grad
    nit = 0
    nitmax = 50000
    alfa = 0.15
    
    while np.linalg.norm(grad) > 1e-5 and nit < nitmax:
        nit = nit + 1
        W = W + (alfa * dir) 
        Ye = np.dot(Xtr,W.T)
        grad = np.dot(((h/var) * (Ytr-Ye)).T, Xtr/N)
        dir = grad 
        
    diff = Ytr - Ye
    soma = 0

    for i in range(m):
        soma = soma + (h[i] * np.dot(diff[i,:],diff[i,:].T))
    var = max(0.05, (1/ns) * soma/np.sum(h))
    
    return W,var

In [10]:
def mistura(Xtr, Ytr, m):
    Ntr = Xtr.shape[0]
    ne = Xtr.shape[1]
    ns = Ytr.shape[1]

    ##add bias
    Xtr = np.concatenate((Xtr, np.ones((Ntr,1))), axis=1)
    ne = ne + 1
    
    ## Inicializa rede gating
    Wg = np.random.rand(m,ne)
    
    # Inicializa especialistas
    W = {}
    var = list(range(m))
    for i in range(m):
        W[i] = np.random.rand(ns,ne)
        var[i] = 1
        
    ##calcula saida
    Yg = softmax(np.dot(Xtr,Wg.T))
    
    Ye = {}
    for i in range(m):
        Ye[i] = np.dot(Xtr,W[i].T)
    Ym = np.zeros((Ntr,ns))
    for i in range(m):
        Yge = Yg[:,i].reshape(Ntr,1)
        Ym = Ym + Ye[i]*Yge
        
    ##calculo da verossimilhanca    
    Py = np.zeros((Ntr,m))
    for i in range(m):
        Yaux = Ye[i]
        for j in range(Ntr):
            diff = Ytr[j,:]- Yaux[j,:]
            Py[j,i] = np.exp(np.dot(-diff,diff.T)/(2*var[i]))
            
    likelihood = np.sum(np.log(np.sum(Yg*Py, axis = 1, keepdims = True)))
    likelihood_ant = 0
    nit = 0
    nitmax = 100
    
    while np.abs(likelihood-likelihood_ant)>1e-3 and nit < nitmax:
        nit = nit + 1
        #Passo E
        haux = Yg*Py
        h = haux / np.dot(np.sum(haux, axis=1, keepdims=True), np.ones((1,m)))
        ##Passo M
        Wg = maximiza_gating(Wg,Xtr,m,h)
        for i in range(m):
            W[i],var[i] = maximiza_expert(W[i],var[i],Xtr,Ytr,h[:,i].reshape(Ntr,1))
        likehood_ant = likelihood


        ##calcula a saida
        Yg = softmax(np.dot(Xtr,Wg.T))
        Ye = {}
        for i in range(m):
            Ye[i] = np.dot(Xtr,W[i].T)
        Ym = np.zeros((Ntr,ns))
        for i in range(m):
            Yge = Yg[:,i].reshape(Ntr,1)
            Ym = Ym + Ye[i]*Yge


        ##calculo da funcao de verossimilhanca    
        Py = np.zeros((Ntr,m))
        for i in range(m):
            Yaux = Ye[i]
            for j in range(Ntr):
                diff = Ytr[j,:]- Yaux[j,:]
                Py[j,i] = np.exp(np.dot(-diff,diff.T)/(2*var[i]))

        likelihood = np.sum(np.log(np.sum(Yg*Py, axis = 1, keepdims = True)))
        
    me = {}
    me['gating'] = Wg
    me['expert_W'] = W
    me['expert_var'] = var
    
    return me

In [11]:
me = mistura(Xtr, Ytr, m)

In [12]:
me['gating']

array([[-0.07257774,  0.67078746,  0.04468822,  0.5314381 ],
       [ 0.27605375,  0.21073047,  1.16297441,  0.50755212],
       [ 1.12177825,  0.70963482,  0.07971823,  0.13381973]])

In [13]:
me['expert_W']

{0: array([[ 0.00627051, -0.04879736,  0.09344427,  0.68881673]]),
 1: array([[ 0.44527098,  0.77453593, -0.54335696,  0.03501006]]),
 2: array([[-0.41963676, -0.15996698,  0.03008576, -0.45956735]])}

In [14]:
##Obtendo saida

Ntr = Xtr.shape[0]
ne = Xtr.shape[1]
ns = Ytr.shape[1]

##add bias
Xtr = np.concatenate((Xtr, np.ones((Ntr,1))), axis=1)
ne = ne + 1


Wg = me['gating']
W = me['expert_W']


##calcula a saida
Yg = softmax(np.dot(Xtr,Wg.T))
Ye = {}
for i in range(m):
    Ye[i] = np.dot(Xtr,W[i].T)
Ym = np.zeros((Ntr,ns))
for i in range(m):
    Yge = Yg[:,i].reshape(Ntr,1)
    Ym = Ym + Ye[i]*Yge

In [15]:
np.sum(Ym -Ytr)/Ntr

3.859717805132787e-06

In [16]:
Ytr

array([[ 1.967004e-01],
       [ 9.226179e-01],
       [-7.024477e-01],
       [ 1.313450e-02],
       [ 9.996550e-01],
       [-9.986201e-01],
       [-9.944842e-01],
       [-9.779978e-01],
       [-9.129594e-01],
       [-6.669898e-01],
       [ 1.102493e-01],
       [ 9.756902e-01],
       [-7.811937e-01],
       [ 7.368374e-01],
       [-7.920459e-01],
       [ 6.854684e-01],
       [-6.613866e-01],
       [ 7.413643e-01],
       [-7.045470e-01],
       [ 3.774754e-01],
       [-5.027848e-01],
       [ 8.011111e-01],
       [-9.508619e-01],
       [ 8.028059e-01],
       [-9.034285e-01],
       [ 8.607385e-01],
       [-6.614806e-01],
       [ 7.858180e-01],
       [-4.361511e-01],
       [ 3.226026e-01],
       [-1.527395e-01],
       [-5.657270e-02],
       [-3.412753e-01],
       [ 5.894084e-01],
       [-5.012277e-01],
       [ 6.487506e-01],
       [-4.689965e-01],
       [ 5.276395e-01],
       [-5.677613e-01],
       [ 6.190836e-01],
       [-6.698887e-01],
       [ 8.04515

In [17]:
Ym

array([[ 0.06322691],
       [ 0.14726841],
       [-0.07426668],
       [ 0.16052153],
       [-0.18817823],
       [-0.31668888],
       [ 0.19769309],
       [-0.18902917],
       [ 0.2701785 ],
       [ 0.25335508],
       [ 0.18051211],
       [-0.13050503],
       [-0.23689207],
       [ 0.18572853],
       [-0.4128122 ],
       [ 0.43963277],
       [-0.40487081],
       [ 0.44025968],
       [-0.37062527],
       [ 0.42326263],
       [-0.24727318],
       [ 0.39573202],
       [-0.34402456],
       [ 0.3800789 ],
       [-0.5354102 ],
       [ 0.46001308],
       [-0.53947585],
       [ 0.46257893],
       [-0.38178376],
       [ 0.42687322],
       [-0.15240312],
       [ 0.31228945],
       [ 0.07547515],
       [ 0.23183964],
       [-0.18807534],
       [ 0.35619718],
       [-0.26133334],
       [ 0.39153042],
       [-0.20452498],
       [ 0.37861518],
       [-0.27978052],
       [ 0.4024877 ],
       [-0.4091637 ],
       [ 0.42610097],
       [-0.44123702],
       [ 0

In [18]:
##Obtendo saida

Nv = Xv.shape[0]
ne = Xv.shape[1]
ns = Yv.shape[1]

##add bias
Xv = np.concatenate((Xv, np.ones((Nv,1))), axis=1)
ne = ne + 1


Wg = me['gating']
W = me['expert_W']


##calcula a saida
Yg = softmax(np.dot(Xv,Wg.T))
Ye = {}
for i in range(m):
    Ye[i] = np.dot(Xv,W[i].T)
Ym = np.zeros((Nv,ns))
for i in range(m):
    Yge = Yg[:,i].reshape(Nv,1)
    Ym = Ym + Ye[i]*Yge

In [19]:
np.sum(Ym -Yv)/Nv

-0.008488912882185916

In [20]:
Yv

array([[ 0.6019081],
       [-0.6992859],
       [ 0.699651 ],
       [-0.6319539],
       [ 0.8119542],
       [-0.8726618],
       [ 0.9397416],
       [-0.8332047],
       [-0.3884601],
       [ 0.6981976],
       [ 0.0250403],
       [ 0.998746 ],
       [-0.994987 ],
       [-0.9799983],
       [-0.9207932],
       [-0.6957203],
       [ 0.0319465],
       [ 0.9979588],
       [-0.9918437],
       [-0.967508 ],
       [-0.8721433],
       [-0.5212678],
       [ 0.4565597],
       [ 0.5831065],
       [ 0.3199737],
       [ 0.7952337],
       [-0.2647933],
       [ 0.859769 ],
       [-0.4784054],
       [ 0.5422565],
       [ 0.4119157],
       [ 0.6606509],
       [ 0.1270808],
       [ 0.9677009],
       [-0.8728902],
       [-0.5238745],
       [ 0.451111 ],
       [ 0.5929977],
       [ 0.2967074],
       [ 0.8239295],
       [-0.3577196],
       [ 0.7440734],
       [-0.1072906],
       [ 0.9769775],
       [-0.9089699],
       [-0.6524527],
       [ 0.1486109],
       [ 0.95

In [21]:
np.round(Ym,3)

array([[ 0.387],
       [-0.349],
       [ 0.428],
       [-0.375],
       [ 0.428],
       [-0.387],
       [ 0.413],
       [-0.562],
       [ 0.455],
       [-0.169],
       [-0.373],
       [ 0.35 ],
       [-0.164],
       [ 0.193],
       [-0.188],
       [ 0.255],
       [ 0.19 ],
       [-0.096],
       [-0.303],
       [ 0.191],
       [-0.188],
       [ 0.242],
       [ 0.131],
       [-0.283],
       [ 0.153],
       [ 0.133],
       [ 0.   ],
       [ 0.164],
       [-0.239],
       [ 0.35 ],
       [-0.228],
       [ 0.243],
       [ 0.054],
       [ 0.153],
       [-0.114],
       [ 0.166],
       [-0.175],
       [-0.281],
       [ 0.147],
       [ 0.136],
       [-0.014],
       [ 0.163],
       [-0.24 ],
       [ 0.363],
       [-0.213],
       [ 0.259],
       [-0.177],
       [-0.148],
       [-0.2  ],
       [ 0.164],
       [-0.472],
       [ 0.451],
       [-0.252],
       [ 0.256],
       [ 0.113],
       [ 0.139],
       [ 0.151],
       [ 0.121],
       [ 0.226